In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout ,UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import matplotlib.pyplot  as plt
import random


2023-06-06 12:01:03.079853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 12:01:03.305528: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-06 12:01:03.345998: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 12:01:03.346050: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [5]:
SIZE=750 # input size
batch_size=1
datagen=ImageDataGenerator(rescale=1/255) # scale betw 0 to 1


In [4]:
train_generator=datagen.flow_from_directory(
    'images/',
    target_size=(SIZE,SIZE),
    batch_size=1,
    class_mode='input'
) 

Found 10 images belonging to 2 classes.


In [7]:
validation_generator=datagen.flow_from_directory(
    'validation/',
    target_size=(SIZE,SIZE),
    batch_size=1,
    class_mode='input'
) 

Found 3 images belonging to 2 classes.


In [8]:
validation_generator=datagen.flow_from_directory(
    'no_normal/',
    target_size=(SIZE,SIZE),
    batch_size=2,
    class_mode='input'
) 

Found 1 images belonging to 1 classes.


In [24]:
train_generator.image_shape

(28, 28, 3)

### autoencoding


In [25]:
model=Sequential()  # model.add(Conv2D(64,(3,3)# filter 3 x 3 ,activation='relu',padding='same' # shift ,stride (jumping ),input_shape=(SIZE,SIZE)))
# encode
model.add(Conv2D(64,(4,4) ,activation='relu',padding='same',input_shape=(train_generator.image_shape)))
model.add(MaxPooling2D(2,padding='same'))# no learning just doing shrinking the matrix
model.add(Conv2D(32,(3,3) ,activation='relu',padding='same'))
model.add(MaxPooling2D(2,padding='same'))
model.add(Conv2D(16,(3,3) ,activation='relu',padding='same'))
model.add(MaxPooling2D(2,padding='same'))

#decode
model.add(Conv2D(16,(3,3) ,activation='relu',padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32,(3,3) ,activation='relu',padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(64,(3,3) ,activation='relu',padding='same'))
model.add(UpSampling2D((2,2)))

model.add(Conv2D(3,(3,3),activation='sigmoid',padding='same'))
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 28, 28, 64)        3136      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 14, 14, 32)        18464     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 7, 7, 16)          4624      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 16)        

In [26]:
history=model.fit(
    train_generator,
    steps_per_epoch=500 // 2,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=1,
    shuffle=True
)

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "/home/shahid/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/shahid/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/shahid/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/shahid/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/shahid/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/shahid/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/shahid/anaconda3/l

In [11]:
iter(train_generator)

In [15]:
train_generator[0][0][0][0]

array([[0.7568628 , 0.76470596, 0.81568635],
       [0.7725491 , 0.7803922 , 0.82745105],
       [0.78823537, 0.7960785 , 0.8431373 ],
       [0.7960785 , 0.80392164, 0.85098046],
       [0.8000001 , 0.81568635, 0.85098046],
       [0.8078432 , 0.8235295 , 0.8588236 ],
       [0.81568635, 0.8313726 , 0.86666673],
       [0.8235295 , 0.8431373 , 0.86666673],
       [0.8431373 , 0.8470589 , 0.86666673],
       [0.8470589 , 0.85098046, 0.8705883 ],
       [0.854902  , 0.8588236 , 0.87843144],
       [0.86274517, 0.86666673, 0.8862746 ],
       [0.854902  , 0.86666673, 0.8862746 ],
       [0.86274517, 0.8745099 , 0.8941177 ],
       [0.87843144, 0.882353  , 0.90196085],
       [0.8862746 , 0.89019614, 0.909804  ],
       [0.89019614, 0.8941177 , 0.91372555],
       [0.89019614, 0.8941177 , 0.91372555],
       [0.9058824 , 0.909804  , 0.92549026],
       [0.9058824 , 0.909804  , 0.92549026],
       [0.8980393 , 0.91372555, 0.92549026],
       [0.8941177 , 0.91372555, 0.92549026],
       [0.

In [17]:
train_generator[0]

AttributeError: 'tuple' object has no attribute 'shape'